Imports

In [32]:
import gurobipy as gb
import pandas as pd
import numpy as np
import ast


Column Generation from Pre-processed Data

In [33]:
def indices_to_list(indices, length):
    return [1 if i in indices else 0 for i in range(length+1)][1:]

In [37]:
pre_data = pd.read_csv('combinations.tsv',sep='\t')
pre_data['Chairs'] = pre_data['Chairs'].apply(ast.literal_eval)
pre_data['Members'] = pre_data['Members'].apply(ast.literal_eval)
pre_data['Chairs'] = pre_data['Chairs'].apply(lambda x: indices_to_list(x,10))
pre_data['Members'] = pre_data['Members'].apply(lambda x: indices_to_list(x,30))
pre_data['a'] = pre_data['Members'] + pre_data['Chairs']
pre_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16344393 entries, 0 to 16344392
Data columns (total 5 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   Day      int64 
 1   Time     int64 
 2   Members  object
 3   Chairs   object
 4   a        object
dtypes: int64(2), object(3)
memory usage: 623.5+ MB


In [41]:
data = pre_data[['Day','Time','a']]

The meat and potatoes of it all...

In [ ]:
m = gb.Model('YvoneStrat')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-27
